In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 96 kB 3.3 MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9568 sha256=67572118269389771abb8a5d4619ad52ddc79753fb5d57fe6e7c0b319a34e30c
  Stored in directory: /root/.cache/pip/wheels/38/5c/e0/13003e68c17f403af40b92a24d20171b95fef13b0fdaba833c
Successfully built kt-legacy


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
import math

train = pd.read_csv('./data/Training.csv')
test = pd.read_csv('./data/Testing.csv')

x_train = train.drop('prognosis', axis=1)
symptoms = x_train.columns
x_train = np.array(x_train)
x_test = test.drop('prognosis', axis=1)
x_test = np.array(x_test)

y_train = train['prognosis']
y_train = pd.get_dummies(y_train)
causes = y_train.columns
y_train = np.array(y_train)
y_test = pd.get_dummies(test['prognosis'])
y_test = np.array(y_test)

In [ ]:
train

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_ urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,...,internal_itching,toxic_look_(typhos),depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic _patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,fluid_overload.1,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
4916,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Urinary tract infection
4918,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,Psoriasis


In [ ]:
x_train

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 1, 1]])

In [ ]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
symptoms

Index(['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing',
       'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity',
       'ulcers_on_tongue',
       ...
       'pus_filled_pimples', 'blackheads', 'scurring', 'skin_peeling',
       'silver_like_dusting', 'small_dents_in_nails', 'inflammatory_nails',
       'blister', 'red_sore_around_nose', 'yellow_crust_ooze'],
      dtype='object', length=132)

In [ ]:
causes

Index(['(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne',
       'Alcoholic hepatitis', 'Allergy', 'Arthritis', 'Bronchial Asthma',
       'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis',
       'Common Cold', 'Dengue', 'Diabetes ', 'Dimorphic hemmorhoids(piles)',
       'Drug Reaction', 'Fungal infection', 'GERD', 'Gastroenteritis',
       'Heart attack', 'Hepatitis B', 'Hepatitis C', 'Hepatitis D',
       'Hepatitis E', 'Hypertension ', 'Hyperthyroidism', 'Hypoglycemia',
       'Hypothyroidism', 'Impetigo', 'Jaundice', 'Malaria', 'Migraine',
       'Osteoarthristis', 'Paralysis (brain hemorrhage)',
       'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis',
       'Typhoid', 'Urinary tract infection', 'Varicose veins', 'hepatitis A'],
      dtype='object')

In [ ]:
# choosing number of layers and neurons: https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw

input_layer_units = len(x_train[0])
num_classes = y_train.shape[1]
step = round((input_layer_units - num_classes)/15)
num_layers = 2
min_units = math.ceil(num_classes/num_layers)
max_units = math.floor(input_layer_units/num_layers)

print('input layer units:', input_layer_units)
print('num classes:', num_classes)
print('step:', step)
print('min units:', min_units)
print('max_units:', max_units)

input layer units: 132
num classes: 41
step: 6
min units: 20
max_units: 66


In [ ]:
# choosing activation function: https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
# choosing optimizer: https://towardsdatascience.com/7-tips-to-choose-the-best-optimizer-47bb9c1219e
# choosing loss function:
  # https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
  # https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy
# choosing learning rate range: https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0

def model_builder(hp):
  # tune the number of units in the hidden layer
  # Choose an optimal value between 20-66
  hp_units = hp.Int('units', 
                    min_value=min_units, 
                    max_value=max_units, 
                    step=step)

  model = keras.Sequential([
    layers.Dense(units=hp_units, input_dim=input_layer_units, activation='relu'),
    layers.Dense(units=hp_units, activation='relu'),
    layers.Dense(units=num_classes, activation='softmax')
  ])

  model.summary()

  # tune the learning rate for the optimizer
  # Choose an optimal value between 1e-1 to 1e-15
  learning_rate_values = np.linspace(1e-1, 1e-15, 15)
  hp_learning_rate = hp.Choice('learning_rate', values=learning_rate_values.tolist())

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss='categorical_crossentropy',
      metrics=['accuracy'])

  return model

In [ ]:
# hyperparameter tuning: https://www.tensorflow.org/tutorials/keras/keras_tuner

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=15,
                     factor=3,
                     directory='./',
                     project_name='prognosis_model_progress')

stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=0)
tuner.search(x_train, y_train, epochs=25, validation_split=0.2, callbacks=[stop_early])

# get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f'''
Optimal number of units per hidden layer: {best_hps.get('units')} 
Optimal learning rate: {best_hps.get('learning_rate')}
''')

Trial 30 Complete [00h 00m 01s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 34s
INFO:tensorflow:Oracle triggered exit

Optimal number of units per hidden layer: 38 
Optimal learning rate: 0.06428571428571464



In [ ]:
# build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 38)                5054      
_________________________________________________________________
dense_1 (Dense)              (None, 38)                1482      
_________________________________________________________________
dense_2 (Dense)              (None, 41)                1599      
Total params: 8,135
Trainable params: 8,135
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
123/123 [==============================] - 1s 3ms/step - loss: 0.2981 - accuracy: 0.9294 - val_loss: 0.0159 - val_accuracy: 1.0000
Epoch 2/50
123/123 [==============================] - 0s 2ms/step - loss: 0.1757 - accuracy: 0.9743 - val_loss: 0.4661 - val_accuracy: 0.9512
Epoch 3/50
123/123 [==============================] - 0s 2ms/step - loss: 0.3216 - accuracy: 0.

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# retrain the model with best hyperparameters
hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)
hypermodel.save('prognosis_model.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 38)                5054      
_________________________________________________________________
dense_1 (Dense)              (None, 38)                1482      
_________________________________________________________________
dense_2 (Dense)              (None, 41)                1599      
Total params: 8,135
Trainable params: 8,135
Non-trainable params: 0
_________________________________________________________________
None
123/123 [==============================] - 1s 3ms/step - loss: 0.2685 - accuracy: 0.9322 - val_loss: 0.1108 - val_accuracy: 0.9756


In [ ]:
del hypermodel
hypermodel = keras.models.load_model('prognosis_model.h5')
eval_result = hypermodel.evaluate(x_test, y_test)
print('[test loss, test accuracy]:', eval_result)

2/2 [==============================] - 0s 5ms/step - loss: 0.1108 - accuracy: 0.9756
[test loss, test accuracy]: [0.1107613667845726, 0.9756097793579102]


##Manual Test

In [ ]:
# transforming binary data from a test row to symptoms, similar to the format of the data after extracting it from a user message
test_symptoms = []
for i in np.where(x_test[0]==1):
  test_symptoms = symptoms[i]

test_symptoms

Index(['itching', 'skin_rash', 'nodal_skin_eruptions', 'dischromic _patches'], dtype='object')

In [ ]:
# transform symptom list to binary data that the model can interpret
new_patient = np.zeros(len(symptoms))

for i, symptom in enumerate(symptoms):
  if symptom in test_symptoms:
    new_patient[i] = 1    

new_patient = [new_patient]
new_patient

[array([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]

In [ ]:
cause_probabilities = hypermodel.predict(np.array(new_patient))
cause_probabilities = cause_probabilities[0]

probable_causes = cause_probabilities.argsort()[::-1] # sort cause probabilities in descending order
causes_and_probabilites = {}

for cause in probable_causes:
  prob = cause_probabilities[cause]
  
  cause_has_significant_chance = prob > .05
  if cause_has_significant_chance:
    causes_and_probabilites[causes[cause]] = str(prob*100) + '%'
    
print('The following may be cause(s) of your symptoms with corresponding chances:')
for cause, probability in causes_and_probabilites.items():
  print('\t', cause, probability)

The following may be cause(s) of your symptoms with corresponding chances:
	 Fungal infection 100.0%


In [ ]:
ground_truth = causes[np.where(y_test[0]==1)]
ground_truth[0]

'Fungal infection'

In [ ]:
!zip -r prognosis_model_progress.zip .
from google.colab import files
files.download('prognosis_model_progress.zip')
files.download('prognosis_model.h5')

updating: .config/ (stored 0%)
updating: .config/gce (stored 0%)
updating: .config/.last_survey_prompt.yaml (stored 0%)
updating: .config/active_config (stored 0%)
updating: .config/.last_update_check.json (deflated 22%)
updating: .config/config_sentinel (stored 0%)
updating: .config/configurations/ (stored 0%)
updating: .config/configurations/config_default (deflated 15%)
updating: .config/.last_opt_in_prompt.yaml (stored 0%)
updating: .config/logs/ (stored 0%)
updating: .config/logs/2021.07.16/ (stored 0%)
updating: .config/logs/2021.07.16/13.20.21.584309.log (deflated 55%)
updating: .config/logs/2021.07.16/13.19.23.269640.log (deflated 91%)
updating: .config/logs/2021.07.16/13.20.22.328313.log (deflated 54%)
updating: .config/logs/2021.07.16/13.19.58.237112.log (deflated 86%)
updating: .config/logs/2021.07.16/13.20.04.172291.log (deflated 54%)
updating: .config/logs/2021.07.16/13.19.42.311157.log (deflated 54%)
updating: data/ (stored 0%)
updating: data/Testing.csv (deflated 85%)
up